In [ ]:
# RUN PROGRAM IN LOCAL ENVIRONMENT

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.facebook.com/timnasindonesia/posts/pfbid02DHN2nWaCSDXvaSXjS4SbqJiGVsbGhyqvaVyjcJfo6Wy1oKGVuz8xQCXRUvbd9WhUl')

time.sleep(5)

# Comment mode from 'Most Relevant' to 'All Comments'
try:
    most_relevant_mode = driver.find_element(
        By.XPATH, 
        '//span[contains(text(), "Most relevant")]'
    )

    ActionChains(driver).move_to_element(most_relevant_mode).click().perform()

    time.sleep(5)

    all_comments_mode = driver.find_element(
        By.XPATH,
        '//span[contains(text(), "All comments")]'
    )

    all_comments_mode.click()

    time.sleep(5)

    print('Successfully changed comment mode')
except Exception as e:
    print('Failed to change comment mode:', str(e))

# Scroll pop up post to show comments
try:
    pop_up_post = driver.find_element(
        By.XPATH, 
        '//div[@class="xb57i2i x1q594ok x5lxg6s x78zum5 xdt5ytf x6ikm8r x1ja2u2z x1pq812k x1rohswg xfk6m8 x1yqm8si xjx87ck xx8ngbg xwo3gff x1n2onr6 x1oyok0e x1odjw0f x1iyjqo2 xy5w88m"]',
    )

    last_height = 0
    while True:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', pop_up_post)

        time.sleep(10)

        new_height = driver.execute_script('return arguments[0].scrollHeight;', pop_up_post)
        
        if new_height == last_height:
            break
        
        last_height = new_height

    print('Scrolling inside pop up post is complete')
except Exception as e:
    print('Failed to scroll in pop up post:', str(e))

time.sleep(5)

# Click 'See More' to expand comments
try:
    while True:
        see_more_buttons = driver.find_elements(
            By.XPATH, 
            '//div[contains(text(), "See more")]',
        )
        if not see_more_buttons:
            break

        for button in see_more_buttons:
            try:
                driver.execute_script("arguments[0].click();", button)

                time.sleep(5)
            except Exception as e:
                print("Gagal mengklik 'See more':", str(e))

    print('All comments have been expanded')
except Exception as e:
    print('Failed to expand comment:', str(e))

time.sleep(5)

# Scraping comments
comments = driver.find_elements(By.XPATH, '//div[@class="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs"]')

# Save comments to list
comments_text = [comment.text for comment in comments]

# Set comments into dataframe
comments_df = pd.DataFrame(comments_text, columns=['comment'])
print(comments_df.shape)

In [ ]:
# Show comments into dataframe
pd.set_option('display.max_colwidth', None)
comments_df.sample(5)

(2545, 1)


,comment
150,"Perjuangan belum selesai sampai disini, tetap semangat timnasQ\napapun hasilnya Aq akan slalu mendukungmu. Terima kasih atas perjuanganmu"
571,Awowok ku menangis
782,Di dalam pssi sepertinya ada mafia
293,"Bpk erik tohir gimana ,luarbiasa pelatih baru,nya pak ,bisa kalah 5:1 mantap,\nSty memang hebat anti mafia tulus berkorban demi timnas"
1213,yang masih bilang proses mana suaranya


In [ ]:
# Save comments into CSV format
comments_df.to_csv('facebook_comments.csv', index=False, encoding='utf-8')